In [1]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

# Dossiers
ANNOTATIONS_DIR = 'annotations'
IMAGES_DIR = 'images'
YOLO_LABELS_DIR = 'labels'
os.makedirs(YOLO_LABELS_DIR, exist_ok=True)

# Classes à détecter
CLASSES = ['person', 'helmet', 'vest']

def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[2]) / 2.0
    y = (box[1] + box[3]) / 2.0
    w = box[2] - box[0]
    h = box[3] - box[1]
    return (x * dw, y * dh, w * dw, h * dh)

for file in os.listdir(ANNOTATIONS_DIR):
    if not file.endswith(".xml"):
        continue

    tree = ET.parse(os.path.join(ANNOTATIONS_DIR, file))
    root = tree.getroot()
    image_name = root.find('filename').text
    image_path = os.path.join(IMAGES_DIR, image_name)
    image = Image.open(image_path)
    w, h = image.size

    yolo_filename = os.path.splitext(image_name)[0] + ".txt"
    yolo_path = os.path.join(YOLO_LABELS_DIR, yolo_filename)

    with open(yolo_path, 'w') as f:
        for obj in root.findall('object'):
            cls = obj.find('name').text
            if cls not in CLASSES:
                continue
            cls_id = CLASSES.index(cls)
            xmlbox = obj.find('bndbox')
            b = [int(xmlbox.find(x).text) for x in ['xmin', 'ymin', 'xmax', 'ymax']]
            bb = convert((w, h), b)
            f.write(f"{cls_id} {' '.join(f'{a:.6f}' for a in bb)}\n")

print("✅ Conversion terminée. Les fichiers .txt ont été créés dans /labels.")

✅ Conversion terminée. Les fichiers .txt ont été créés dans /labels.
